In [10]:
import os 
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.output_parsers import StrOutputParser 
from langchain.prompts import ChatPromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings

In [5]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4")

In [18]:
model.invoke("Bist du nicht chat gpt 3?")
# returns JSON data with model output and metadata 

AIMessage(content='Ja, ich bin ein Modell von OpenAI und basiere auf der Technologie GPT-3. Ich bin ein KI-gesteuerter Chatbot.', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 17, 'total_tokens': 51}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6d14c275-5ae8-47aa-b098-af97da169ef9-0')

In [5]:
response = model.invoke("Erkläre LLMs in drei Sätzen")
response.content

'LLMs, kurz für "Large Language Models", sind fortgeschrittene KI-Systeme, die darauf trainiert sind, menschliche Sprache zu verstehen und zu generieren. Sie basieren auf einer Architektur namens Transformer, die große Mengen an Textdaten verarbeitet, um Muster, Kontext und Sprachnuancen zu lernen. Diese Modelle werden in vielfältigen Anwendungen genutzt, von der Textgenerierung und Übersetzung bis hin zur Beantwortung von Fragen und der Unterstützung bei der Textbearbeitung, indem sie fähig sind, kohärente und kontextbezogene Antworten zu generieren.'

In [6]:
parser = StrOutputParser() # Initialisierung eines Objekts zum Extrahieren des Strings 
chain = model | parser # Output des Model wird in parser Objekt gepiped
chain.invoke("Erkläre LLMs in drei Sätzen")

'LLMs, oder "Large Language Models", sind fortschrittliche KI-Systeme, die darauf trainiert sind, menschliche Sprache zu verstehen und zu generieren. Sie basieren auf neuronalen Netzwerken, insbesondere auf Architekturen wie Transformer, die große Mengen an Textdaten verarbeiten, um Muster und Kontexte zu lernen. LLMs werden in verschiedenen Anwendungen verwendet, darunter Textgenerierung, Übersetzung, Zusammenfassung und andere sprachbasierte Aufgaben, um Benutzern zu helfen, Informationen effizienter zu verarbeiten und zu verstehen.'

In [7]:
template = """
Beantworte die Frage basierend auf dem unten gegebenen Kontext. 
Wenn das Beantworten der Frage nicht möglich ist durch den gegebenen Kontekt, antworte "Ich weiß es nicht". 

Kontext: {context}

Frage: {question}
"""

prompt = ChatPromptTemplate.from_template(template) # Template Prompt Objekt erzeugen


In [8]:
model.invoke(prompt.format(context="Es gibt auschließlich weiße Schwäne.", question = "Gibt es schwarze Schwäne?")).content


'Ich weiß es nicht.'

Du heißt Max.


In [9]:
chain = prompt | model | parser
chain.invoke({
    "context": "Es gibt ausschließlich weiße Schwäne.", 
    "question": "Gibt es schwarze Schwäne?"
})

'Ich weiß es nicht.'

In [10]:
from operator import itemgetter

diff_prompt = ChatPromptTemplate.from_template(
    "Übersetze {answer} in {language}"
)

diff_chain = (
    # dic um Input zu definieren, dies wird dann in diff_prompt gepiped
    {"answer": chain, # Antwort wird aus der anderen Chain als Input genommen
     "language": itemgetter("language")} # Sprache aus Dictonary
    | diff_prompt | model | parser 
)

diff_chain.invoke(
    {
        "context": "Alle Schwäne sind weiß, außer in Australien. Dort gibt es schwarze.",
        "question": "Gibt es schwarze Schwäne?", 
        "language": "Italienisch"
    }
)

'Das Wort "Ja" auf Italienisch ist "Sì".'

In [4]:
embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Gibt es schwarze Schwäne?")

print(f"Embedding length: {len(embedded_query)}") # Dimension der Embedding Liste

Embedding length: 1536
[0.00901121263534296, -0.03669295798076368, -0.00873078020641617, -0.025039455366582045, -0.01990443465229751, 0.006879930087054519, -0.0007376917735520665, -0.012351468412199479, -0.009098458010849193, -0.04162856202435205]


In [6]:
sen1 = embeddings.embed_query("Es gibt in Europa nur weiße Schwäne")
sen2 = embeddings.embed_query("Es gibt schwarze Schwäne in Australien")

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

query_sen1_sim = cosine_similarity([embedded_query], [sen1])[0][0]
query_sen2_sim = cosine_similarity([embedded_query], [sen2])[0][0]
query_sen1_sim, query_sen2_sim

(0.8740940461805541, 0.9229002354973087)

In [20]:
import rag
run_id= "25eec295-c3d5-4946-80bf-6e17f19e15af"
client = rag.get_client()
feedback_record = client.create_feedback(
    run_id,
    key = "test",
    score=1,
                )

In [1]:
import os 
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate 
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langsmith import Client
from langchain.callbacks import LangChainTracer

# load .env Variablen 
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# tracing mit Langsmith from Langchain
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "RAG-project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
client = Client()

In [2]:
def send_feedback(run_id,score):
    key =f"feedback_{run_id}"
    client.create_feedback(
        run_id,
        key=key,
        score=score,
        )

In [4]:
send_feedback("217b483c-e520-498d-bb49-d0e0b5a43ff0",1)

In [5]:
list = [True, False]
sum(list)

1